In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from tensorflow.examples.tutorials.mnist import input_data

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
batch_size = 100
num_classes = 10
epochs = 100

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x_train = mnist.train.images
y_train = mnist.train.labels

x_test = mnist.test.images
y_test = mnist.test.labels

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(784,)))
#model.add(Dropout(0.5))
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)
                    #validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 795,010
Trainable params: 795,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
55000/55000 [==============================] - 4s 81us/step - loss: 0.2356 - acc: 0.9319
Epoch 2/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0909 - acc: 0.9723
Epoch 3/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0571 - acc: 0.9823
Epoch 4/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0369 - acc: 0.9890
Epoch 5/100
55000/55000 [==============================] - 2s 32us/step - loss: 0.0263 - acc: 0.9925
Epoch 6/100
55000/55000 [==

55000/55000 [==============================] - 2s 32us/step - loss: 1.3280e-06 - acc: 1.0000
Epoch 74/100
55000/55000 [==============================] - 2s 32us/step - loss: 1.1265e-06 - acc: 1.0000
Epoch 75/100
55000/55000 [==============================] - 2s 32us/step - loss: 8.7117e-07 - acc: 1.0000
Epoch 76/100
55000/55000 [==============================] - 2s 32us/step - loss: 7.1225e-07 - acc: 1.0000
Epoch 77/100
55000/55000 [==============================] - 2s 32us/step - loss: 5.9591e-07 - acc: 1.0000
Epoch 78/100
55000/55000 [==============================] - 2s 32us/step - loss: 5.0589e-07 - acc: 1.0000
Epoch 79/100
55000/55000 [==============================] - 2s 32us/step - loss: 4.3200e-07 - acc: 1.0000
Epoch 80/100
55000/55000 [==============================] - 2s 32us/step - loss: 3.7822e-07 - acc: 1.0000
Epoch 81/100
55000/55000 [==============================] - 2s 32us/step - loss: 3.2740e-07 - acc: 1.0000
Epoch 82/100
55000/55000 [==============================] -

In [24]:
from keras.layers.core import Lambda
from keras import backend as K

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(10,)))
model.add(Lambda(lambda x: K.dropout(x, level=0.5)))
model.add(Dense(512, activation='relu'))
model.add(Lambda(lambda x: K.dropout(x, level=0.5)))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error',
      optimizer=Adam(),
      metrics=['mse'])

X = np.random.randn(1000, 10)
y = np.random.randn(1000, 1)
model.fit(X, y,
            batch_size=100,
            epochs=10,
            verbose=1)

Epoch 1/10
1000/1000 [==============================] - 0s 145us/step - loss: 1.0622 - mean_squared_error: 1.0622
Epoch 2/10
1000/1000 [==============================] - 0s 27us/step - loss: 1.0116 - mean_squared_error: 1.0116
Epoch 3/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9756 - mean_squared_error: 0.9756
Epoch 4/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9862 - mean_squared_error: 0.9862
Epoch 5/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9622 - mean_squared_error: 0.9622
Epoch 6/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9629 - mean_squared_error: 0.9629
Epoch 7/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9262 - mean_squared_error: 0.9262
Epoch 8/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9445 - mean_squared_error: 0.9445
Epoch 9/10
1000/1000 [==============================] - 0s 27us/step - loss: 0.9299 - mean_squa

In [34]:
np.std([model.predict(X).reshape(-1), model.predict(X).reshape(-1)], axis=0)

array([5.33907153e-02, 6.39157891e-02, 7.77018666e-02, 9.68090817e-02,
       4.66681421e-02, 1.84875667e-01, 2.88804471e-02, 5.69854677e-02,
       9.88591164e-02, 1.36360347e-01, 2.07519829e-01, 9.14089382e-04,
       9.20131430e-02, 7.18265176e-02, 1.33261859e-01, 2.93259680e-01,
       2.13871807e-01, 6.15872145e-02, 1.54801801e-01, 1.42381191e-02,
       1.47582561e-01, 4.69868407e-02, 6.83117099e-03, 2.95405477e-01,
       1.23980917e-01, 3.58434319e-02, 1.61114603e-01, 7.71168917e-02,
       7.70851970e-02, 6.97559565e-02, 8.68657157e-02, 1.01845108e-01,
       2.82934189e-01, 1.77677959e-01, 1.37961239e-01, 2.68434174e-02,
       7.61663169e-02, 1.41134709e-02, 2.01288611e-01, 1.52663171e-01,
       2.26762906e-01, 1.68845773e-01, 7.46788383e-02, 1.08935218e-02,
       9.37108696e-03, 4.22323793e-02, 9.85164940e-02, 1.86700404e-01,
       2.18603890e-02, 1.67545289e-01, 8.42307955e-02, 9.89872441e-02,
       5.50210588e-02, 9.79016051e-02, 7.81806856e-02, 4.06549126e-02,
      

In [17]:
import numpy as np
np.random.randn(100, 10)

array([[ 1.08766576e+00, -1.11143225e-01,  2.99663660e-01,
         2.72185398e-01,  3.17682673e-01,  1.24845960e+00,
        -4.86517395e-01,  2.68588391e-01, -7.61824243e-01,
        -7.41325901e-01],
       [ 7.82488046e-01, -2.34690375e+00, -5.05615939e-01,
        -4.50711166e-01,  4.47639771e-02,  4.73596467e-01,
        -6.25196222e-01,  9.68197563e-01, -1.35818273e-01,
         9.13167066e-01],
       [-3.58699241e-01,  1.37307508e-01, -5.92358939e-01,
        -1.97164471e+00,  4.44127807e-01, -2.86065590e-01,
         8.64629556e-01, -1.84925877e-01,  3.88667591e-01,
         3.94468865e-01],
       [ 2.13688676e-01, -1.26098297e+00, -4.03446785e-01,
         2.10303801e-01,  2.38029216e-01, -8.50251669e-01,
        -5.71056242e-01, -2.11692588e+00,  3.31400818e-01,
         2.44720717e+00],
       [ 6.02967560e-01, -1.25173374e+00,  1.16708042e+00,
        -3.27444388e-02,  1.62968113e+00, -6.74920060e-01,
         2.35064544e+00,  2.19057513e+00, -4.81975739e-01,
        -9.

In [33]:
X.shape[-1]

10